## IBSI Chapter 1 Phase 1 − Radiomic Computations

@Author : [MEDomics consortium](https://github.com/medomics/)

@EMAIL : medomics.info@gmail.com

@REF : [IBSI 1](https://arxiv.org/pdf/1612.07003.pdf)

**STATEMENT**:
This file is part of <https://github.com/MEDomics/MEDomicsLab/>,
a package providing PYTHON programming tools for radiomics analysis.
--> Copyright (C) MEDomicsLab consortium.

This package is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This package is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this package.  If not, see <http://www.gnu.org/licenses/>.

### Introduction


We recommend to take a look at the [MEDimage-tutorial notebook](https://colab.research.google.com/github/MahdiAll99/MEDimage/blob/dev/notebooks/tutorial/MEDimage-Tutorial.ipynb) and the [DataManager-tutorial](https://colab.research.google.com/github/MahdiAll99/MEDimage/blob/dev/notebooks/tutorial/DataManager-Tutorial.ipynb) before running the IBSI tests.

In this notebook we treat the first phase of standardization of image processing and feature computation. In the figure below, we focus on the first part referred as phase 1. We only compute radiomics features from a digital phantom without any processing

<img src="images/Flowchart-of-study-overview.png" alt="range resegmentation example"/>


### Dataset - Digital phantom
In this chapter and in this phase, reference values for features were obtained using a digital image phantom, which is described below. The digital phantom can be found here: https://github.com/theibsi/data_sets/tree/master/ibsi_1_digital_phantom

- The phantom consists of 5 × 4 × 4 (x, y, z) voxels.
- A slice consists of the voxels in (x, y) plane for a particular slice at position z. Slices are therefore stacked in the z direction.
- Voxels are 2.0 × 2.0 × 2.0 mm in size.
- Not all voxels are included in the region of interest. Several excluded voxels are located on the outside of the ROI, and one internal voxel was excluded as well. Voxels excluded from the ROI are shown in blue in figure below.
- Some intensities are not present in the phantom. Notably, grey levels 2 and 5 are absent. 1 is the lowest grey level present in the ROI and 6 the highest.

<img src="images/view-of-the-test-volume.png" alt="IBSI 1 Phase 1 Digital Phantom" style="width:400px;"/>

In [1]:
import os
import sys

MODULE_DIR = os.path.dirname(os.path.abspath('../MEDimage/'))
sys.path.append(os.path.dirname(MODULE_DIR))

from copy import deepcopy
from pathlib import Path

import MEDimage

### Classes initialization
In the IBSI scripts we are going to use the *MEDimage* class and other processing classes like *DataManager* to process the iamging data and to extract the radiomics features.
- ``MEDimage``: Is a Python class that organizes all scan data and many other useful information that is used by the many processing and computing methods. Learn more in the [MEDimage-tutorial]()
- ``DataManager``: A Python class that process all the raw data (DICOM and NIfTI) and convert it to *MEDimage* class objects. Learn more in the [DataManager-tutorial]()

So the first step is to initialize the MEDimage class using the *DataManager*. This class will only need path to where the data is located (either DICOM or NIfTI). We will use NIfTI for this tutorial.

Make sure your folder structure looks like this:

<img src="images/ibsi1-p1-folder-structure.png"/>

Make sure to also download the [Phantom data](https://github.com/theibsi/data_sets/tree/master/ibsi_1_digital_phantom/dicom) (we recommend you use NIfTI files) and place it in Phantom folder. The settings file can be found in the repository and is automatically place in the settings folder.

We now start by initializing our assets (paths, variables...)

In [2]:
path_settings = Path(os.getcwd()) / "settings" # Path to the script settings/configuration folder
# Load script parameters
im_params = MEDimage.utils.json_utils.load_json(path_settings / 'Phase1_settings.json')

Then we'll initialize the ``DataManager`` class and run NIfTIs processing

In [3]:
# Intialize the DataManager class
path_to_niftis = Path(os.getcwd()) / "data" / "Phantom"
dm = MEDimage.wrangling.DataManager(path_to_niftis=path_to_niftis)

# Process the NIfTI scan
MEDinstance = dm.process_all_niftis()[0]  # one-element list/ one scan


--> Scanning all folders in initial directory


100%|██████████| 2/2 [00:00<00:00, 40721.40it/s]


DONE
--> Reading all NIfTI objects (imaging volumes & masks) to create MEDimage classes


100%|██████████| 1/1 [00:00<00:00, 442.20it/s]

DONE


### Image processing

In the MEDimage package, the ``processing`` module offers many methods and functionalities for image processing (interpolation, segmentation...) and will be used in this phase, but we only need to to extract the ROI and replace the excluded values (values outside the ROI) in the image volume with a placeholder (NaN). The intensity and morphological mask are identical in this case (no re-segmentation or interpolation is done here).

In [4]:
# Extraction of ROI mask :
vol_obj_init, roi_obj_init = MEDimage.processing.get_roi_from_indexes(MEDinstance,
                                                           name_roi='{tumor}',
                                                           box_string='full')

# Init processing & computation parameters
MEDinstance.init_params(im_params)

# Morphological Mask :
vol_obj = deepcopy(vol_obj_init)
roi_obj_morph = deepcopy(roi_obj_init)

# Intensity mask Creation :
roi_obj_int = deepcopy(roi_obj_morph)

# Preparation of computation :
MEDinstance.init_ntf_calculation(vol_obj)

# Image volume ROI Extraction :
vol_int_re = MEDimage.processing.roi_extract(
    vol=vol_obj.data, 
    roi=roi_obj_int.data
)

### Non-Texture Features extraction
In this section we extract the following families of features using ``biomarkers`` methods : 

*morphological features, local intensity, statistical, Intensity-based and intensity histogram-based.*

No further image processing is required.

#### Morphological features

Morphological features describe geometric aspects of a region of interest (ROI), such as area and
volume. Morphological features are based on ROI voxel representations of the volume.

In [5]:
morph = MEDimage.biomarkers.morph.extract_all(
            vol=vol_obj.data, 
            mask_int=roi_obj_int.data, 
            mask_morph=roi_obj_morph.data,
            res=MEDinstance.params.process.scale_non_text,
        )

#### Local intensity features

Voxel intensities within a defined neighborhood around a center voxel are used to compute local
intensity features. By definition, local intensity features are calculated in 3D, and not per slice.

In [6]:
local_intensity = MEDimage.biomarkers.local_intensity.extract_all(
            img_obj=vol_obj.data,
            roi_obj=roi_obj_int.data,
            res=MEDinstance.params.process.scale_non_text)

#### Intensity-based statistical features

The intensity-based statistical features describe how intensities within the region of interest (ROI)
are distributed. The features in this set do not require discretization, and may be used to describe
a continuous intensity distribution.

In [7]:
stats = MEDimage.biomarkers.stats.extract_all(
            vol=vol_int_re
        )

#### Intensity histogram features

An intensity histogram is generated by discretizing the original intensity distribution into
intensity bins.

In [8]:
int_hist = MEDimage.biomarkers.intensity_histogram.extract_all(vol=vol_int_re)

#### Intensity-volume histogram features

The (cumulative) intensity-volume histogram (IVH) of the set of voxel intensities in the ROI
intensity mask describes the relationship between discretized intensity and the fraction of the
volume containing at least intensity the same intensity.

In [9]:
int_vol_hist = MEDimage.biomarkers.int_vol_hist.extract_all(
                MEDimg=MEDinstance,
                vol=vol_int_re,
                vol_int_re=vol_int_re, 
                wd=1
        )

### Texture Features extraction
In this section, for each text scale<sup>1</sup> we extract the matrix-based features using the same module ``biomarkers``:

*Grey level co-occurrence based features (GLCM), grey level run length based features (GLRLM), grey level size zone matrix (GLSZM), grey level distance zone matrix (GLDZM), neighborhood grey tone difference matrix (NGTDM) and neighboring grey level dependence matrix (NGLDM).*

After the computation is finished, we update the radiomics structure (update the attributes for results). 

No further image processing is done in this section as well.

<sup>1</sup> For each time we resample the voxel spacing (In this case we resample the voxel spacing one time).

**Note**: For our case (IBSI 1, Phase 1) we only re-sample the voxel spacing one time so the texture features will be calculated one time.

In [10]:
# Intensity mask creation :
roi_obj_int = deepcopy(roi_obj_morph)

# Preparation of computation :
MEDinstance.init_tf_calculation(algo=0, gl=0, scale=0)

# ROI Extraction :
vol_quant_re = MEDimage.processing.roi_extract(
    vol=vol_obj.data, 
    roi=roi_obj_int.data)

#### Grey level co-occurrence based features

The grey level co-occurrence matrix (GLCM) is a matrix that expresses how combinations of
discretized intensities (grey levels) of neighboring pixels, or voxels in a 3D volume, are distributed
along one of the image directions.

In [11]:
glcm = MEDimage.biomarkers.glcm.extract_all(
                    vol=vol_quant_re, 
                    dist_correction=MEDinstance.params.radiomics.glcm.dist_correction)

#### Grey level run length based features
The grey level run length matrix (GLRLM) defines various texture features. Like the grey level co-occurrence matrix, GLRLM also assesses the distribution of
discretized grey levels in an image or in a stack of images. However, whereas GLCM assesses
co-occurrence of grey levels within neighboring pixels or voxels, GLRLM assesses run lengths. A
run length is defined as the length of a consecutive sequence of pixels or voxels with the same grey level along a direction.

In [12]:
glrlm = MEDimage.biomarkers.glrlm.extract_all(
                    vol=vol_quant_re,
                    dist_correction=MEDinstance.params.radiomics.glrlm.dist_correction)

#### Grey level size zone based features

The grey level size zone matrix (GLSZM) counts the number of groups (or zones) of linked voxels.
Voxels are linked if the neighboring voxel has an identical neighboring grey level.

In [13]:
glszm = MEDimage.biomarkers.glszm.extract_all(vol=vol_quant_re)

#### Grey level distance zone based features

The grey level distance zone matrix (GLDZM) counts the number of groups (or zones) of linked
voxels which share a specific neighboring grey level value and possess the same distance to ROI
edge. The GLDZM thus captures the relation between location and grey level.

In [14]:
gldzm = MEDimage.biomarkers.gldzm.extract_all(
                    vol_int=vol_quant_re, 
                    mask_morph=roi_obj_morph.data)

####  Neighbourhood grey tone difference based features

The neighborhood grey tone difference matrix (NGTDM) contains the sum of grey level differences
of pixels/voxels with a discretized grey level and the average discretized grey level of neighboring pixels/voxels within a Chebyshev distance. 

In [15]:
ngtdm = MEDimage.biomarkers.ngtdm.extract_all(
                    vol=vol_quant_re, 
                    dist_correction=MEDinstance.params.radiomics.ngtdm.dist_correction)

####  Neighbouring grey level dependence based features:

The neighbouring grey level dependence matrix (NGLDM) aims to capture the coarseness of the overall
texture and is rotationally invariant.

In [16]:
ngldm = MEDimage.biomarkers.ngldm.extract_all(vol=vol_quant_re)

#### Update the radiomics structure

This can be done by calling the *MEDimage* class method ``update_radiomics``

In [17]:
MEDinstance.update_radiomics(
            int_vol_hist_features=int_vol_hist, 
            morph_features=morph,
            loc_int_features=local_intensity, 
            stats_features=stats, 
            int_hist_features=int_hist,
            glcm_features=glcm, 
            glrlm_features=glrlm, 
            glszm_features=glszm, 
            gldzm_features=gldzm, 
            ngtdm_features=ngtdm, 
            ngldm_features=ngldm
        )

Finally we print the results

In [18]:
from numpyencoder import NumpyEncoder
from json import dumps

print(dumps(MEDinstance.radiomics.image, indent=4, cls=NumpyEncoder))

{
    "morph_3D": {
        "scale2": {
            "Fmorph_vol": 556.3333333333334,
            "Fmorph_approx_vol": 592.0,
            "Fmorph_area": 388.0706298811092,
            "Fmorph_av": 0.6975505629977996,
            "Fmorph_comp_1": 0.04105763998875515,
            "Fmorph_comp_2": 0.5989495056264417,
            "Fmorph_sph_dispr": 1.1863238540244057,
            "Fmorph_sphericity": 0.8429401437117419,
            "Fmorph_asphericity": 0.1863238540244052,
            "Fmorph_com": 0.6715449258791162,
            "Fmorph_diam": 13.114877048604,
            "Fmorph_pca_major": 11.402387266727741,
            "Fmorph_pca_minor": 9.308010776621932,
            "Fmorph_pca_least": 8.535981219598838,
            "Fmorph_pca_elongation": 0.816321228080262,
            "Fmorph_pca_flatness": 0.7486135157421726,
            "Fmorph_v_dens_aabb": 0.8692708333333334,
            "Fmorph_a_dens_aabb": 0.8662290845560473,
            "Fmorph_v_dens_ombb": 0.8692708333333334,
         

You can compare your results with other teams and check your level of consensus using the CSV provided [here](https://ibsi.radiomics.hevs.ch/assets/IBSI-1-submission-table.xlsx)